# Mocap Data Filter Pipeline Plots

## Imports and Global Helper Functions

In [ ]:
import json

from pathlib import Path

import pandas as pd
import numpy as np
import scipy.signal as ss

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps, register_fausans_font

import biopsykit as bp

from empkins_io.sensors.motion_capture.perception_neuron import PerceptionNeuronDataset
from empkins_io.processing.utils.rotations import euler_to_quat, quat_to_euler, rotate_quat

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
register_fausans_font()

plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

## Variable and Path Setup

In [ ]:
deploy_type = "local"

In [ ]:
config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))
base_path = Path(config_dict[deploy_type]["base_path"])
data_path = base_path.joinpath("data_per_subject")

plot_path = Path("../../results/plots")
bp.utils.file_handling.mkdirs([plot_path])

In [ ]:
subject_id = "VP_03"
condition = "tsst"

In [ ]:
mocap_path = data_path.joinpath(f"{subject_id}/{condition}/mocap")
input_path = mocap_path.joinpath("export")
output_path = mocap_path.joinpath("filtered")
bp.utils.file_handling.mkdirs(output_path)

In [ ]:
bvh_file = f"{subject_id}_{condition}.bvh.gz"
calc_file = f"{subject_id}_{condition}.calc.gz"
global_pose_file = f"{subject_id}_{condition}_global_pose.csv.gz"

bvh_path = input_path.joinpath(bvh_file)
calc_path = input_path.joinpath(calc_file)

filter_params = json.load(mocap_path.joinpath(f"{subject_id}_filter_params_{condition}.json").open(encoding="utf-8"))

In [ ]:
dataset = PerceptionNeuronDataset.from_folder(input_path, **filter_params["start_end"])
data_total = dataset.data_as_df(index="time")
data_total.index.name = "Time [s]"

In [ ]:
pos_filter_params = filter_params["pos_filter_params"]
rot_filter_params = filter_params["rot_filter_params"]

## Drift Filtering Pipeline

### Process Data

#### Input Data

In [ ]:
data_in = data_total.loc[:, pd.IndexSlice["bvh", "Hips", "rot", :]]

#### Unwrapping

In [ ]:
data_unwrap = np.unwrap(np.deg2rad(data_in), axis=0)
data_unwrap = pd.DataFrame(data_unwrap, index=data_in.index, columns=data_in.columns)
data_unwrap_plot = np.rad2deg(data_unwrap)

#### Conversion to Quaternions

In [ ]:
data_quat = euler_to_quat(data_unwrap)

#### Approximation of Rotation Drift

In [ ]:
sos = ss.butter(
    N=rot_filter_params[0].get("N", 1),
    Wn=rot_filter_params[0].get("Wn", 0.01),
    fs=dataset.sampling_rate_hz,
    btype="high",
    output="sos",
)
data_filt = ss.sosfiltfilt(sos=sos, x=data_quat, axis=0)
data_filt = pd.DataFrame(data=data_filt, columns=data_quat.columns, index=data_quat.index)

data_drift_approx = data_quat - data_filt

#### Rotation with Approximated Drift

In [ ]:
data_rot = rotate_quat(data_quat, data_drift_approx)
data_rot = pd.DataFrame(data=data_rot, columns=data_quat.columns, index=data_quat.index)

#### Conversion to Euler Angles

In [ ]:
data_euler = np.rad2deg(quat_to_euler(data_rot))

### Plot Pipeline

In [ ]:
data_in.columns = list("yxz")
data_unwrap_plot.columns = list("yxz")
data_quat.columns = list("wxyz")
data_drift_approx.columns = list("wxyz")
data_rot.columns = list("wxyz")
data_euler.columns = list("yxz")

In [ ]:
fig, axs = plt.subplots(ncols=6, nrows=1, figsize=(15, 2.5))
data_iter = [data_in, data_unwrap_plot, data_quat, data_drift_approx, data_rot, data_euler]

legend_kwargs = dict(loc="upper right", fontsize="smaller", handlelength=0.5)

ylabels = [
    "Rotation [°]",
    "Rotation [°]",
    "Quaternion [AU]",
    "Quaternion [AU]",
    "Quaternion [AU]",
    "Rotation [°]",
    "Rotation [°]",
]

titles = [
    "Original\n Euler Angles",
    "Unwrapped\n Euler Angles",
    "Unwrapped\n Quaternions",
    "Approximated\n Rotation Drift",
    "Drift-compensated\n Quaternions",
    "Drift-compensated\n Euler Angles",
]

for data, ylabel, title, ax in zip(data_iter, ylabels, titles, axs):
    data.plot(ax=ax)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(labels, **legend_kwargs)


fig.tight_layout(pad=0.5, w_pad=1.0, rect=(0.01, 0, 0.99, 1))
fig.savefig(plot_path.joinpath("img_rotation_filter_pipeline.pdf"))